In [2]:
import pandas as pd
import pandasql as psql


filePath = r"D:\Program Files\2024\final_capstone\test_salary.xlsx"

if filePath.endswith(".xlsx"):
  df = pd.read_excel(filePath)
  rd_df = pd.read_excel(filePath, sheet_name="RD")
elif filePath.endswith(".csv"):
  df = pd.read_csv(filePath)
  rd_df = pd.read_csv(filePath, sheet_name="RD")
  
  
df['timeIn'] = pd.to_datetime(df['timeIn'])
df['timeOut'] = pd.to_datetime(df['timeOut'])


# # Calculate work hours
# df['totalWorkHours'] = df['timeOut'] - df['timeIn']
# #transform into hours based
# df['totalWorkHours'] = df.apply(lambda row: row['totalWorkHours'].total_seconds() / 3600, axis = 1)

# # Calculate late time
# df['late'] = df.apply(lambda row: (row['timeIn'] - row['workingTime']).total_seconds() / 60 
#                if pd.notnull(row['timeIn']) and pd.notnull(row['timeOut']) and row['timeIn'] > row['workingTime'] else 0, axis=1)

# df['absent'] = df.apply(lambda row: 1 if pd.isna(row['timeIn']) and pd.isna(row['timeOut']) and row['workingTime'] != 'RD' and pd.isna(row['totalWorkHours']) else '', axis = 1)


df

,uuid,name,workingTime,timeIn,timeOut
0,1,Alice Smith,2024-07-15 09:00:00,2024-07-15 09:00:00,2024-07-15 17:00:00
1,1,Alice Smith,2024-07-16 09:00:00,2024-07-16 09:15:00,2024-07-16 17:00:00
2,1,Alice Smith,2024-07-17 09:00:00,NaT,NaT


In [ ]:
july_dates = pd.date_range(start='2024-07-01', end='2024-07-31')
dates_df = pd.DataFrame({"date": july_dates})


#data from uuid & name
test_df = df.drop_duplicates(subset=['uuid'],keep='first')
test_df = test_df[['uuid','name']]

dates_df['key'] = 1
test_df['key'] = 1

result = pd.merge(test_df, dates_df,on="key").drop("key", axis=1)

result

In [6]:
query = '''
SELECT
b.uuid, b.name, DATE(b.date), a.timeIn, a.timeOut from result b
left join df a
on DATE(a.workingTime) = DATE(b.date) and a.uuid = b.uuid
left join rd_df as rd
on DATE(b.workingTime) = DATE(rd.workingTime)
'''

result_sql = psql.sqldf(query,locals())
result_sql

PandaSQLException: (sqlite3.OperationalError) no such table: result
[SQL: 
SELECT
b.uuid, b.name, DATE(b.date), a.timeIn, a.timeOut from result b
left join df a
on DATE(a.workingTime) = DATE(b.date) and a.uuid = b.uuid
left join rd_df as rd
on DATE(b.workingTime) = DATE(rd.workingTime)
]
(Background on this error at: https://sqlalche.me/e/20/e3q8)

In [140]:
print(result.head(5))
result.columns

   uuid         name       date
0     1  Alice Smith 2024-07-01
1     1  Alice Smith 2024-07-02
2     1  Alice Smith 2024-07-03
3     1  Alice Smith 2024-07-04
4     1  Alice Smith 2024-07-05


Index(['uuid', 'name', 'date'], dtype='object')

In [139]:
df.columns

Index(['uuid', 'name', 'workingTime', 'timeIn', 'timeOut'], dtype='object')

In [4]:
rd_df

,uuid,name,workingTime,status
0,1,Alice Smith,2024-07-21,RD
1,1,Alice Smith,2024-07-28,RD
